In [ ]:
import logging
import os
from pathlib import Path

import geopandas as gpd
import rasterio
from shapely.geometry import MultiPolygon, Polygon
from shapely.geometry import box, mapping

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s %(message)s",
)

In [ ]:
study_area = "CORDILLERA BLANCA"  # "CORDILLERA BLANCA" or "CORDILLERA VILCABAMBA"

In [ ]:
input_path = Path(f"data/{study_area}")

polygons = []

for raster_folder in os.listdir(input_path):
    sample_image = os.listdir(input_path / f"{raster_folder}")[0]
    sample_image = input_path / f"{raster_folder}/{sample_image}"

    with rasterio.open(sample_image) as src:
        bbox = src.bounds

        poly = box(bbox[0], bbox[1], bbox[2], bbox[3])
        poly_geojson = mapping(poly)
        polygons.append(poly_geojson)

In [ ]:
polygons_shapely = [Polygon(poly["coordinates"][0]) for poly in polygons]
multi_polygon = MultiPolygon(polygons_shapely)
external_contour = multi_polygon.convex_hull
external_contour

In [ ]:
gdf = gpd.GeoDataFrame(geometry=[external_contour])
gdf.crs = src.crs
gdf = gdf.to_crs("EPSG:4326")
gdf.to_file(f"{study_area}.shp")

logging.info(f"Bounding box saved in {study_area}.shp")